<a href="https://colab.research.google.com/github/cawllol/Deep-learning/blob/master/BusinessCase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Exploring the data set and identifying predictors
#we can use this same preprocess any data set to pre process for two classes
#importing libraries
import numpy as np
import tensorflow as tf
from sklearn import preprocessing

In [0]:
raw_csv_data=np.loadtxt('/content/original (2).csv',delimiter=',')
unscaled_inputs_all=raw_csv_data[:,1:-1]
targets_all=raw_csv_data[:,-1]

In [0]:
#banlance the dataset
#if we sum all targets we will get the number of targets that are 1s
num_one_targets=int(np.sum(targets_all))
zero_targets_counter=0
indices_to_remove=[]
for i in range(targets_all.shape[0]):#length of vector
  if targets_all[i]==0:
    zero_targets_counter +=1
    if zero_targets_counter>num_one_targets:
      indices_to_remove.append(i)
''' if the target at position is 0 and the number of zeroes is bigger than 
 the number of 1s,we want to take note of that index'''
unscaled_inputs_equal_priors=np.delete(unscaled_inputs_all,indices_to_remove,axis=0)
 #it deletes an object along an axis
targets_equal_prior=np.delete(targets_all,indices_to_remove,axis=0)

In [0]:
#scalling input
scaled_inputs=preprocessing.scale(unscaled_inputs_equal_priors)

In [0]:
#shuffle data
# taking indices from axis 0 and place them in a variable  
shuffled_indices=np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)
shuffled_inputs=scaled_inputs[shuffled_indices]
shuffled_targets=targets_equal_prior[shuffled_indices]

In [9]:
#train,test,validation
samples_count=shuffled_inputs.shape[0]
train_samples_count=int(0.8*samples_count)
validation_samples_count=int(0.1*samples_count)
test_samples_count=samples_count-train_samples_count-validation_samples_count
train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]

print(np.sum(train_targets), train_samples_count, np.sum(train_targets) / train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets) / validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets) / test_samples_count)

1765.0 3579 0.49315451243364067
237.0 447 0.5302013422818792
235.0 448 0.5245535714285714


In [0]:
#Save the three datasets in *.npz
np.savez('Audiobooks_data_train', inputs=train_inputs, targets=train_targets)
np.savez('Audiobooks_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('Audiobooks_data_test', inputs=test_inputs, targets=test_targets)

In [0]:
#creating ML algorithm
#loading saved data
npz=np.load('/content/Audiobooks_data_train.npz')
train_inputs=npz['inputs'].astype(np.float)
train_targets=npz['targets'].astype(np.int)

npz=np.load('/content/Audiobooks_data_validation.npz')
validation_inputs,validation_targets=npz['inputs'].astype(np.float),npz['targets'].astype(np.int)

npz=np.load('/content/Audiobooks_data_test.npz')
test_inputs,test_targets=npz['inputs'].astype(np.float),npz['targets'].astype(np.int)

In [14]:
#Modelling
input_size = 10
output_size = 2
hidden_layer_size = 50

model = tf.keras.Sequential([
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(output_size, activation='softmax')   
                            ])
#optimizer & loss function
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

batch_size=100
max_epochs=100
early_stopping=tf.keras.callbacks.EarlyStopping(patience=2)
model.fit(train_inputs,train_targets,
          batch_size=batch_size,epochs=max_epochs,
          callbacks=[early_stopping],
          validation_data=(validation_inputs,validation_targets),verbose=2)

Epoch 1/100
36/36 - 0s - loss: 0.5971 - accuracy: 0.6636 - val_loss: 0.5250 - val_accuracy: 0.7427
Epoch 2/100
36/36 - 0s - loss: 0.4797 - accuracy: 0.7541 - val_loss: 0.4579 - val_accuracy: 0.7450
Epoch 3/100
36/36 - 0s - loss: 0.4220 - accuracy: 0.7877 - val_loss: 0.4191 - val_accuracy: 0.7740
Epoch 4/100
36/36 - 0s - loss: 0.3927 - accuracy: 0.7966 - val_loss: 0.4015 - val_accuracy: 0.7785
Epoch 5/100
36/36 - 0s - loss: 0.3739 - accuracy: 0.8033 - val_loss: 0.3833 - val_accuracy: 0.7830
Epoch 6/100
36/36 - 0s - loss: 0.3643 - accuracy: 0.8036 - val_loss: 0.3703 - val_accuracy: 0.7987
Epoch 7/100
36/36 - 0s - loss: 0.3545 - accuracy: 0.8064 - val_loss: 0.3682 - val_accuracy: 0.7987
Epoch 8/100
36/36 - 0s - loss: 0.3510 - accuracy: 0.8080 - val_loss: 0.3593 - val_accuracy: 0.8031
Epoch 9/100
36/36 - 0s - loss: 0.3455 - accuracy: 0.8083 - val_loss: 0.3551 - val_accuracy: 0.8143
Epoch 10/100
36/36 - 0s - loss: 0.3404 - accuracy: 0.8175 - val_loss: 0.3509 - val_accuracy: 0.8098
Epoch 11/

In [15]:
test_loss,test_accuracy=model.evaluate(test_inputs,test_targets)

14/14 [==============================] - 0s 2ms/step - loss: 0.3264 - accuracy: 0.8125
